<a href="https://colab.research.google.com/github/Wnjoki/PILyTQOJRh9c4klH/blob/main/Potential_Talent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
!pip install transformers
!pip install sentence_transformers

In [2]:
# data processing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import re
import string
from collections import Counter

#Text-Processing
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


#models
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
import torch






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load  and Explore Dataset

In [3]:
#load the data
talent_df = pd.read_csv("/potential-talents - Aspiring human resources - seeking human resources.csv")


In [4]:
talent_df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [5]:
talent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


In [6]:
#Statistical analysis of categorical columns
talent_df.describe(include=object)

,job_title,location,connection
count,104,104,104
unique,52,41,33
top,2019 C.T. Bauer College of Business Graduate (...,Kanada,500+
freq,7,12,44


In [7]:
talent_df.shape

(104, 5)

In [8]:
len(talent_df['id'].unique())

104

In [9]:
talent_df = talent_df.set_index('id')

talent_df.head()

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [10]:
#count dublicated rows
talent_df.loc[talent_df.duplicated(), :]
print(talent_df.duplicated().sum())

51


In [11]:
#drop dublicated rows
talent_df.drop_duplicates(inplace=True)
talent_df.shape

(53, 4)

In [12]:
#drop fit column

talent_df.drop('fit', axis=1, inplace=True)

In [13]:
#Verify if ther are some null values

talent_df.isnull().sum()

job_title     0
location      0
connection    0
dtype: int64

No null values in the dataset

#Data Cleansing / Text Preprocessing

In [14]:
nlp = spacy.load("en_core_web_sm")

def pre_process(text, stem="None"):

    final_text = ""

    # Remove all the special characters
    text = re.sub(r'\W', ' ', str(text))

    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)

    # Make text lower case
    text = text.lower()

    # Remove stop words
    text = text.split()
    Stopwords = set(stopwords.words('english'))

    text_filtered = [word for word in text if not word in Stopwords]

    # Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer()
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_text = ' '.join(text_stemmed)

    # Removing whitespace from the start of the string
    final_text = final_text.lstrip()

    # Substituting multiple spaces with single space
    final_text = re.sub(r'\s+', ' ', final_text, flags=re.I)

    return final_text

In [15]:
talent_df['job_title'] = talent_df['job_title'].apply(pre_process, stem='Spacy')

In [16]:
#Example after text preprocessing
#print(talent_df.iloc[0]['job_title'])
talent_df.iloc[0]['job_title']

'bauer college business graduate magna cum laude aspire human resource professional'

In [17]:
#Frequency distribution
plot_dist = talent_df['connection'] .value_counts().sort_index()
fig = px.bar(plot_dist,title='Frequency Distribution for all connections',text_auto=True)
fig.show()

500+ have the highest values

In [18]:
plot_dist = talent_df['location'] .value_counts().sort_index()
fig = px.bar(plot_dist,title='Frequency Distribution for all locations',text_auto=True)
fig.show()

Houstine Texax Area has the highest number of candidates.

In [19]:
#value count of the job titles
talent_df.job_title.value_counts()

aspire human resource professional                                                                2
bauer college business graduate magna cum laude aspire human resource professional                1
lead official western illinois university                                                         1
senior human resource business partner heil environmental                                         1
aspire human resource professional energetic team focus leader                                    1
hr manager endemol shine north america                                                            1
human resource professional world leader gis software                                             1
rrp brand portfolio executive jti japan tobacco international                                     1
information system specialist programmer love datum organization                                  1
bachelor science biology victoria university wellington                                           1


Some of the job_title have some abbreviations to be expanded and also spacing to adjust.

In [20]:
#replace abreviations by there real texts
talent_df.replace({'job_title' : { 'chro' : 'chief human resources officer', 'svp' : 'senior vice president'
        ,'gphr' : 'global professional in human resources','hris' : 'human resources information system'
        , 'csr' : 'corporate social responsibility', 'sphr' : 'Senior Professional in Human Resources'
        , 'hr' : 'human resources', 'rrp': 'recommended retail price'}}, regex=True, inplace=True)

In [21]:
#word counts
words_counts = Counter()
for i in talent_df.job_title:
    for word in i.split(" "):
        words_counts[word] += 1

print('There are', len(words_counts) ,'words in the job title column')
words_counts.most_common()

There are 186 words in the job title column


[('human', 39),
 ('resource', 34),
 ('aspire', 13),
 ('professional', 11),
 ('seek', 10),
 ('manager', 7),
 ('university', 6),
 ('student', 6),
 ('management', 6),
 ('business', 5),
 ('generalist', 5),
 ('resources', 5),
 ('specialist', 4),
 ('position', 4),
 ('senior', 3),
 ('information', 3),
 ('system', 3),
 ('opportunity', 3),
 ('retail', 3),
 ('director', 3),
 ('major', 3),
 ('college', 2),
 ('english', 2),
 ('coordinator', 2),
 ('officer', 2),
 ('in', 2),
 ('internship', 2),
 ('staff', 2),
 ('north', 2),
 ('america', 2),
 ('leader', 2),
 ('bauer', 1),
 ('graduate', 1),
 ('magna', 1),
 ('cum', 1),
 ('laude', 1),
 ('native', 1),
 ('teacher', 1),
 ('epik', 1),
 ('program', 1),
 ('korea', 1),
 ('people', 1),
 ('development', 1),
 ('ryan', 1),
 ('advisory', 1),
 ('board', 1),
 ('member', 1),
 ('celal', 1),
 ('bayar', 1),
 ('humber', 1),
 ('chapman', 1),
 ('vice', 1),
 ('president', 1),
 ('chief', 1),
 ('marketing', 1),
 ('communication', 1),
 ('corporate', 1),
 ('social', 1),
 ('respo

Most frequent words ('human', 39),
 ('resource', 34),
 ('aspire', 13),
 ('professional', 11),
 ('seek', 10)

#Ranking/Score candidates based on keywords.

In [22]:
#create a df with relevant columns

df = talent_df[['job_title', 'connection']]
df.head()

,job_title,connection
id,,
1,bauer college business graduate magna cum laud...,85
2,native english teacher epik english program korea,500+
3,aspire human resource professional,44
4,people development coordinator ryan,500+
5,advisory board member celal bayar university,500+


#doc2vec embeddings

In [23]:
# Doc2Vec Embeddings
tagged_data = [TaggedDocument(words=title.split(), tags=[str(i)]) for i, title in enumerate(df['job_title'])]


In [24]:
# Train Doc2Vec model
model = Doc2Vec(tagged_data, vector_size=100, window=2, min_count=1, workers=4, epochs=20)


In [25]:
# Doc2Vec embedding for "Aspiring human resources" (w1)
w1 = "Aspiring human resources"
w1_vector = model.infer_vector(w1.split())


In [26]:
#  cosine similarity between w1 and job_title embeddings
cos_similarities1= []
for i, title in enumerate(df['job_title']):
    title_words = title.split()
    similarity = cosine_similarity([model.infer_vector(title_words)], [w1_vector])[0][0]
    cos_similarities1.append(similarity)

In [27]:
df['cos_similarities1'] = cos_similarities1

In [28]:
# Convert "connection" column to numeric format
df['connection'] = df.loc[:, 'connection'].str.replace('+', '').astype(float)


<ipython-input-28-acb48d79fc21>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [29]:
# Scale "connection" column between 0 and 1
scaler = MinMaxScaler()
df['scaled_connection'] = scaler.fit_transform(df[['connection']])


In [30]:
# Calculating ranking
df['ranking'] = 0.8 * df['cos_similarities1'] + 0.2 * df['scaled_connection']


In [31]:
df = df.sort_values(by='ranking', ascending=False)
df

,job_title,connection,cos_similarities1,scaled_connection,ranking
id,,,,,
101,human resource generalist loparex,500.0,0.255760,1.000000,0.404608
89,director human resource ey,349.0,0.199702,0.697395,0.299241
104,director administration excellence log,500.0,0.123923,1.000000,0.299138
27,aspire human resource management student seek ...,500.0,0.062199,1.000000,0.249759
78,human resource generalist schwan,500.0,0.041129,1.000000,0.232903
71,human resource generalist scottmadden inc,500.0,0.038277,1.000000,0.230621
77,human resource conflict management policy proc...,409.0,0.054052,0.817635,0.206769
81,senior human resource business partner heil en...,455.0,0.030005,0.909820,0.205968
68,human resource specialist luxottica,500.0,-0.018951,1.000000,0.184839


#Bert Embeddings

In [32]:
# BERT Embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')

In [33]:
#generate BERT embeddings
def get_bert_embedding(text):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model_bert(input_ids)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0).detach().numpy()
    return embeddings

In [34]:
# BERT embedding for "Aspiring human resources" (w1)
w1_embedding = get_bert_embedding(w1)

print(w1_embedding[:3])

[ 0.30581912 -0.08324541  0.24562995]


In [35]:
w1_embedding.shape

(768,)

In [36]:
#job_title BERT embeddings
df['bert_embedding'] = df['job_title'].apply(get_bert_embedding)

print(df['bert_embedding'][:3])

id
101    [0.016921341, -0.19194953, 0.009764032, 0.2993...
89     [-0.23601876, -0.058882218, -0.20357768, 0.164...
104    [-0.22222485, -0.06335052, -0.27392873, 0.0145...
Name: bert_embedding, dtype: object


In [37]:
df['bert_embedding'].shape

(53,)

In [38]:
# Cosine similarity between w1 and job_title BERT embeddings
df['bertCosine_similarity'] = df['bert_embedding'].apply(lambda x: cosine_similarity([x], [w1_embedding])[0][0])

In [39]:
# Ranking based on weighted sum of cosine_similarity and scaled_connection
df['ranking2'] = 0.8 * df['bertCosine_similarity'] + 0.2 * df['scaled_connection']


In [40]:
# Sort the dataframe
df = df.sort_values(by='ranking2', ascending=False)

df.head()

,job_title,connection,cos_similarities1,scaled_connection,ranking,bert_embedding,bertCosine_similarity,ranking2
id,,,,,,,,
8,human resources senior specialist,500.0,-0.054214,1.0,0.156628,"[-0.15310206, -0.013271034, -0.02504923, -0.05...",0.785617,0.828493
67,human resource staff recruit professional,500.0,-0.027456,1.0,0.178035,"[0.20598105, 0.26471734, 0.0620079, 0.21260397...",0.777806,0.822245
68,human resource specialist luxottica,500.0,-0.018951,1.0,0.184839,"[0.055227414, -0.11013625, 0.02690214, 0.26329...",0.758234,0.806587
4,people development coordinator ryan,500.0,-0.118538,1.0,0.105170,"[-0.22670178, -0.28873459, -0.3249328, 0.07421...",0.757261,0.805808
104,director administration excellence log,500.0,0.123923,1.0,0.299138,"[-0.22222485, -0.06335052, -0.27392873, 0.0145...",0.757059,0.805647


In [41]:
df.drop('bert_embedding',axis=1, inplace=True)

#ELMO


In [42]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.compat.v1

tf.compat.v1.disable_eager_execution()

In [43]:
#Load the pre trained Elmo module from TensorFlow Hub:

elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)


In [44]:
#Extract ELMO vectors embeddings
def elmo_vectors(x):

 embeddings=elmo(x, signature="default", as_dict=True)["elmo"]
	# using the GPU for accelerated calculations
 with tensorflow.compat.v1.Session() as sess:
    sess.run(tensorflow.compat.v1.global_variables_initializer())
    sess.run(tensorflow.compat.v1.tables_initializer())
    # return average of ELMo features
    return sess.run(tensorflow.compat.v1.reduce_mean(embeddings,1))


In [45]:
# ELMO embedding for "Aspiring human resources" (w1)
w1 = ["Aspiring human resources"]
w1_embeddings =  elmo_vectors(w1)

w1_embeddings.shape


(1, 1024)

In [46]:
w1_embeddings[0]

array([-0.05690173, -0.08409411, -0.21298318, ...,  0.2026697 ,
        0.16399111, -0.2432735 ], dtype=float32)

In [47]:
df = df.reset_index()


In [48]:
#elmo embeddings for job_title
corpus = df['job_title']

elmo_embeddings=[]
print (len(corpus))
for i in range(len(corpus)):
	 #print (corpus[i])
    elmo_embeddings.append(elmo_vectors([corpus[i]])[0])


53


In [49]:
elmo_embeddings =  np.array(elmo_embeddings)

elmo_embeddings.shape


(53, 1024)

In [50]:
similarities = cosine_similarity(elmo_embeddings, w1_embeddings)

df['elmoCosine_similarity'] = similarities

In [51]:
# Ranking based on weighted sum of cosine_similarity and scaled_connection
df['ranking3'] = 0.8 * df['elmoCosine_similarity'] + 0.2 * df['scaled_connection']


In [52]:
# Sort the dataframe
df = df.sort_values(by='ranking3', ascending=False)

df.head()

,id,job_title,connection,cos_similarities1,scaled_connection,ranking,bertCosine_similarity,ranking2,elmoCosine_similarity,ranking3
8,101,human resource generalist loparex,500.0,0.255760,1.0,0.404608,0.695778,0.756623,0.584995,0.667996
1,67,human resource staff recruit professional,500.0,-0.027456,1.0,0.178035,0.777806,0.822245,0.581809,0.665447
12,78,human resource generalist schwan,500.0,0.041129,1.0,0.232903,0.681570,0.745256,0.579400,0.663520
0,8,human resources senior specialist,500.0,-0.054214,1.0,0.156628,0.785617,0.828493,0.578555,0.662844
6,27,aspire human resource management student seek ...,500.0,0.062199,1.0,0.249759,0.730953,0.784763,0.535604,0.628483


From the  above, ELMO is more effective than doc2vec . It's performance is because it's word vectors successfully address the issue of polysemous. ELMo word representations take the entire input sentence into equation for calculating the word embeddings.

BERT technique is more effective for ranking the talents and would be adopted. It's performance is because it's pretrained large corpus of unlabelled text.

In [53]:
df = df.sort_values(by='ranking2', ascending=False)

df.head()

,id,job_title,connection,cos_similarities1,scaled_connection,ranking,bertCosine_similarity,ranking2,elmoCosine_similarity,ranking3
0,8,human resources senior specialist,500.0,-0.054214,1.0,0.156628,0.785617,0.828493,0.578555,0.662844
1,67,human resource staff recruit professional,500.0,-0.027456,1.0,0.178035,0.777806,0.822245,0.581809,0.665447
2,68,human resource specialist luxottica,500.0,-0.018951,1.0,0.184839,0.758234,0.806587,0.517311,0.613848
3,4,people development coordinator ryan,500.0,-0.118538,1.0,0.105170,0.757261,0.805808,0.321483,0.457187
4,104,director administration excellence log,500.0,0.123923,1.0,0.299138,0.757059,0.805647,0.313564,0.450851


In [54]:
#Top  candidates with highest ranking scores
top_candidates = df.head(10)

In [55]:
# Print the top candidates
top_candidates

,id,job_title,connection,cos_similarities1,scaled_connection,ranking,bertCosine_similarity,ranking2,elmoCosine_similarity,ranking3
0,8,human resources senior specialist,500.0,-0.054214,1.000000,0.156628,0.785617,0.828493,0.578555,0.662844
1,67,human resource staff recruit professional,500.0,-0.027456,1.000000,0.178035,0.777806,0.822245,0.581809,0.665447
2,68,human resource specialist luxottica,500.0,-0.018951,1.000000,0.184839,0.758234,0.806587,0.517311,0.613848
3,4,people development coordinator ryan,500.0,-0.118538,1.000000,0.105170,0.757261,0.805808,0.321483,0.457187
4,104,director administration excellence log,500.0,0.123923,1.000000,0.299138,0.757059,0.805647,0.313564,0.450851
5,75,nortia staff seek human resource payroll admin...,500.0,-0.096018,1.000000,0.123186,0.744905,0.795924,0.441120,0.552896
6,27,aspire human resource management student seek ...,500.0,0.062199,1.000000,0.249759,0.730953,0.784763,0.535604,0.628483
7,89,director human resource ey,349.0,0.199702,0.697395,0.299241,0.771484,0.756666,0.530155,0.563603
8,101,human resource generalist loparex,500.0,0.255760,1.000000,0.404608,0.695778,0.756623,0.584995,0.667996
9,71,human resource generalist scottmadden inc,500.0,0.038277,1.000000,0.230621,0.693905,0.755124,0.469920,0.575936


#Re-ranking Candidates after starring

After starring a candidate(s) by entering the id, the job titles of that candidate(s) becomes the keyword and then the pointwise technique  would be used to re-rank the talent list to select the potential talents.

Re-rank using the job title starred as keyword and finding its cosine similarity to all the other job titles.
For this task Bert Embeddings vectors will be used.

In [56]:
df.set_index('id', inplace=True)


In [57]:
index = df.index
print('Candidate Ids list')
for i in index:
    print(i,end=' ')

Candidate Ids list
8 67 68 4 104 75 27 89 101 71 94 10 78 28 81 13 2 77 103 12 97 69 3 99 85 74 82 66 6 76 73 84 83 72 7 92 102 79 100 80 88 5 98 70 1 86 87 90 96 11 91 95 93 

In [58]:
starred_id = input("Please, enter id of starred candidate(s): ")

Please, enter id of starred candidate(s): 27


In [59]:
# Build BERT_base model and get Bert vectors
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('bert-base-nli-mean-tokens')
rerank_key = df[df.index.isin([int(starred_id)])]['job_title'].tolist()
bert_vectors_rerank = bert_model.encode(list(df['job_title']))
rerank_vectors = bert_model.encode(rerank_key)

In [60]:
# Rerank score
RerankScore = []
for i in range(len(bert_vectors_rerank)) :
    re_rank_score = cosine_similarity(np.reshape(bert_vectors_rerank[i],(1,-1)), np.reshape(rerank_vectors[0],(1,-1)))
    RerankScore.append(re_rank_score[0][0])

df['Rerank Score'] = RerankScore

In [61]:
df.sort_values(by=['Rerank Score'],ascending=False)

,job_title,connection,cos_similarities1,scaled_connection,ranking,bertCosine_similarity,ranking2,elmoCosine_similarity,ranking3,Rerank Score
id,,,,,,,,,,
27,aspire human resource management student seek ...,500.0,0.062199,1.000000,0.249759,0.730953,0.784763,0.535604,0.628483,1.000000
73,aspire human resource manager seek internship ...,7.0,0.006623,0.012024,0.007703,0.759906,0.610330,0.611331,0.491470,0.889547
100,aspire human resource manager graduating may s...,103.0,-0.062533,0.204409,-0.009145,0.673046,0.579319,0.421266,0.377895,0.714228
7,student humber college aspire human resource g...,61.0,0.002135,0.120240,0.025756,0.712176,0.593789,0.546142,0.460962,0.710891
67,human resource staff recruit professional,500.0,-0.027456,1.000000,0.178035,0.777806,0.822245,0.581809,0.665447,0.707067
92,seek employment opportunity within customer se...,64.0,-0.054443,0.126253,-0.018304,0.706317,0.590304,0.379591,0.328924,0.695936
90,undergraduate research assistant styczynski lab,155.0,-0.049920,0.308617,0.021787,0.512272,0.471541,0.356551,0.346965,0.684900
95,student westfield state university,57.0,-0.119369,0.112224,-0.073050,0.511733,0.431831,0.307310,0.268293,0.626093
75,nortia staff seek human resource payroll admin...,500.0,-0.096018,1.000000,0.123186,0.744905,0.795924,0.441120,0.552896,0.621064


The result above shows  the ranked list based on BERT's model being reranked successfully. The starred candidate with id 27 is labeled as 1. Thus our model achieves the goals of this project.



#Ranking Aligorithm

In [62]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split


In [63]:
df1=df.copy()

df1.head()

,job_title,connection,cos_similarities1,scaled_connection,ranking,bertCosine_similarity,ranking2,elmoCosine_similarity,ranking3,Rerank Score
id,,,,,,,,,,
8,human resources senior specialist,500.0,-0.054214,1.0,0.156628,0.785617,0.828493,0.578555,0.662844,0.422910
67,human resource staff recruit professional,500.0,-0.027456,1.0,0.178035,0.777806,0.822245,0.581809,0.665447,0.707067
68,human resource specialist luxottica,500.0,-0.018951,1.0,0.184839,0.758234,0.806587,0.517311,0.613848,0.436933
4,people development coordinator ryan,500.0,-0.118538,1.0,0.105170,0.757261,0.805808,0.321483,0.457187,0.509865
104,director administration excellence log,500.0,0.123923,1.0,0.299138,0.757059,0.805647,0.313564,0.450851,0.419342


In [64]:

# Create a new column 'Target' and assign 0 to all rows
df1['Target'] = 0

df1 = df1.reset_index()
# Assign 1 to the first ten rows of the 'Target' column
df1.loc[:9, 'Target'] = 1

# Print the modified data frame
df1.head()

,id,job_title,connection,cos_similarities1,scaled_connection,ranking,bertCosine_similarity,ranking2,elmoCosine_similarity,ranking3,Rerank Score,Target
0,8,human resources senior specialist,500.0,-0.054214,1.0,0.156628,0.785617,0.828493,0.578555,0.662844,0.422910,1
1,67,human resource staff recruit professional,500.0,-0.027456,1.0,0.178035,0.777806,0.822245,0.581809,0.665447,0.707067,1
2,68,human resource specialist luxottica,500.0,-0.018951,1.0,0.184839,0.758234,0.806587,0.517311,0.613848,0.436933,1
3,4,people development coordinator ryan,500.0,-0.118538,1.0,0.105170,0.757261,0.805808,0.321483,0.457187,0.509865,1
4,104,director administration excellence log,500.0,0.123923,1.0,0.299138,0.757059,0.805647,0.313564,0.450851,0.419342,1


In [65]:
print(df1.columns)

Index(['id', 'job_title', 'connection', 'cos_similarities1',
       'scaled_connection', 'ranking', 'bertCosine_similarity', 'ranking2',
       'elmoCosine_similarity', 'ranking3', 'Rerank Score', 'Target'],
      dtype='object')


In [67]:
#drop some colums to create a new training dataset
coln = ['id', 'job_title', 'connection',  'ranking', 'ranking2','ranking3', 'Rerank Score',]

df1.drop(coln, axis=1, inplace = True)

df1.head()

,cos_similarities1,scaled_connection,bertCosine_similarity,elmoCosine_similarity,Target
0,-0.054214,1.0,0.785617,0.578555,1
1,-0.027456,1.0,0.777806,0.581809,1
2,-0.018951,1.0,0.758234,0.517311,1
3,-0.118538,1.0,0.757261,0.321483,1
4,0.123923,1.0,0.757059,0.313564,1


In [68]:
#get the input feature(X) and Target(y) variables

X = df1.iloc[:, :-1]


y = df1.iloc[:, -1]

# Print X and Y
print("Input features (X):")
print(X.shape)
print("\nTarget variable (Y):")
print(y.shape)

Input features (X):
(53, 4)

Target variable (Y):
(53,)


In [69]:
#split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [70]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(42, 4)
(11, 4)
(42,)
(11,)


In [71]:
query_train = [X_train.shape[0]]
query_test = [X_test.shape[0]]

In [72]:
# Creating the ranker object
ranker = lgb.LGBMRanker(
                    objective="lambdarank",
                    boosting_type = "gbdt",
                    n_estimators = 5,
                    importance_type = "gain",
                    metric= "ndcg",
                    num_leaves = 10,
                    learning_rate = 0.05,
                    max_depth = -1,
                    min_child_weight=0.001,
                    min_child_samples=10,
                    label_gain =[i for i in range(max(y_train.max(), y_test.max()) + 1)])

# Training the model
ranker.fit(
      X=X_train,
      y=y_train,
      group=query_train,
      eval_set=[(X_train, y_train),(X_test, y_test)],
      eval_group=[query_train, query_test],
      eval_at=[4, 8])

[1]	training's ndcg@4: 0.39038	training's ndcg@8: 0.337257	valid_1's ndcg@4: 1	valid_1's ndcg@8: 1
[2]	training's ndcg@4: 0.80481	training's ndcg@8: 0.873529	valid_1's ndcg@4: 1	valid_1's ndcg@8: 1
[3]	training's ndcg@4: 1	training's ndcg@8: 1	valid_1's ndcg@4: 1	valid_1's ndcg@8: 1
[4]	training's ndcg@4: 1	training's ndcg@8: 1	valid_1's ndcg@4: 1	valid_1's ndcg@8: 1
[5]	training's ndcg@4: 1	training's ndcg@8: 1	valid_1's ndcg@4: 1	valid_1's ndcg@8: 1


LGBMRanker(importance_type='gain', label_gain=[0, 1], learning_rate=0.05,
           metric='ndcg', min_child_samples=10, n_estimators=5, num_leaves=10,
           objective='lambdarank')

In [73]:
#predict the model
test_pred = ranker.predict(X_test)

In [74]:
#test the model
X_test["predicted_ranking"] = test_pred
X_test.sort_values("predicted_ranking", ascending=False)

,cos_similarities1,scaled_connection,bertCosine_similarity,elmoCosine_similarity,predicted_ranking
2,-0.018951,1.000000,0.758234,0.517311,0.286103
10,-0.131229,0.829659,0.731498,0.486509,0.034825
19,-0.040466,1.000000,0.614055,0.370247,-0.078717
30,0.006623,0.012024,0.759906,0.611331,-0.122654
31,-0.060129,0.098196,0.732777,0.524690,-0.181211
34,0.002135,0.120240,0.712176,0.546142,-0.200811
39,-0.154646,0.102204,0.687556,0.403102,-0.401608
51,-0.119369,0.112224,0.511733,0.307310,-0.464859
32,-0.195211,0.535070,0.617416,0.387002,-0.464859
46,-0.103039,0.078156,0.569897,0.291741,-0.464859


In [75]:
X_test.drop("predicted_ranking",axis=1, inplace=True)

#Hyperparameter tuning

In [78]:
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import make_scorer, ndcg_score

param_grid = {
    'num_leaves': [5, 10, 15, 20],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [5, 10, 15],
    'subsample': [0.8, 0.9, 1.0],
    'max_depth': [-1],
    'min_gain_to_split':[1,5]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(ranker, param_grid, cv=3, verbose=1, n_jobs=-1, scoring=make_scorer(ndcg_score, needs_proba=True))

# Perform the grid search
#error_score='raise'
grid_search.fit(X_train, y_train, group=np.ones(len(X_train)))

# Get the best parameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)



Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': -1, 'min_gain_to_split': 1, 'n_estimators': 5, 'num_leaves': 5, 'subsample': 0.8}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning:

One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan

In [79]:
# Evaluate the best model on the test set
test_predictions = best_model.predict(X_test)
accuracy = np.mean(test_predictions == y_test)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Test Accuracy: 90.91%


After the hyperparameter training the model achieved an accuracy of 90.91% on test data